## This notebook does the whole-brain voxel-wise GLM fitting

We use FSL FEAT. We first make all event files and a file with volume-wise confounds

In [ ]:
import pandas as pd
import numpy as np
import glob

# for masking
from nilearn.input_data import NiftiMasker
import nibabel as nib
import itertools
import multiprocessing as mp

import os
import shutil

# 1. First, make event files

In [ ]:
events = pd.read_pickle('./derivatives/all_subjects_events.pkl')
confounds = pd.read_pickle('./derivatives/all_subjects_confounds.pkl')
events['onset'] -= 1.5     # slice time correction

In [ ]:
events

In [5]:
def get_sub_n(sub, max_sub_ds1=19):
    if sub > max_sub_ds1:
        sub -= max_sub_ds1
        ds = 2
    else:
        ds = 1
    return ds, sub

#### Events of interest

In [4]:
include_events = ['easy', 'hard', 'error', 'response_left', 'response_right', 'cue_left', 'cue_neutral', 'cue_right']

for (subject, run), events_ in events.groupby(level=[0,1]):
    ds, sub = get_sub_n(subject)    
    output_dir = './derivatives/ds-{ds}/feat_files/sub-{sub}/run-{run}/evs/'.format(**{'ds':str(ds).zfill(2), 'sub':str(sub).zfill(2), 'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    events_['weight'] = 1
    for i, ev in enumerate(include_events):
        events_.loc[events_['event_type']==ev, ['onset', 'duration', 'weight']].to_csv(output_dir+'{}.txt'.format(ev), sep='\t', index=False, header=False)

/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


#### Confounds

In [5]:
for (subject, run), confounds_ in confounds.groupby(level=[0,1]):
    ds, sub = get_sub_n(subject)
    output_dir = './derivatives/ds-{ds}/feat_files/sub-{sub}/run-{run}/evs/'.format(**{'ds':str(ds).zfill(2), 'sub':str(sub).zfill(2), 'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    confounds_.to_csv(output_dir+'confounds.txt', sep='\t', index=False, header=False)

# 2. Mask functional data and fix header (set TR = 3)

For some reason, the fmriprep output (or perhaps already the sourcedata / input) has the TR set wrongly in the header of the nifti files. Here, we correct this using `fslmerge`. Furthermore, we apply a brain mask to the functional data.

In [7]:
def mask_img(img, mask):
    masker = NiftiMasker(mask)
    data_masked = masker.fit_transform(img)
    data_masked_epispace = masker.inverse_transform(data_masked)
    return data_masked_epispace

def mask_data_and_fix_header(arg):
    sub, run = arg
    if sub == 23:  # does not exist
        return 0
    if sub == 6 and run == 3: # does not exist
        return 0    
    if sub == 26 and run == 3: # does not exist
        return 0    
    
    ds, sub = get_sub_n(sub)
    input_dir = './derivatives/ds-{}/fmriprep/sub-{}/func'.format(str(ds).zfill(2), str(sub).zfill(2))
    output_dir = './derivatives/ds-{}/feat_files/sub-{}/run-{}/func'.format(str(ds).zfill(2), 
                                                                            str(sub).zfill(2), 
                                                                            run)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    img_fn = 'sub-{}_task-randomdotmotion_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(str(sub).zfill(2), str(run).zfill(2))
    input_fn = os.path.join(input_dir, img_fn)
    mask_fn = input_fn.replace('desc-preproc_bold', 'desc-brain_mask')
    output_fn = os.path.join(output_dir, img_fn)
    
    if os.path.exists(output_fn):
        return 0
    
    print(input_fn)
    masked_data = mask_img(input_fn, mask_fn)
    nib.save(masked_data, output_fn)
    
    # make sure header is OK
    import subprocess
    return_code = subprocess.run(["fslmerge", "-tr", output_fn, output_fn, "3"])

    return 0

subjects = np.arange(1, 35)
runs = [1,2,3]

to_run = itertools.product(subjects, runs)
to_run_list = list(to_run)
to_run_list.pop(to_run_list.index((6,3)))
to_run_list.pop(to_run_list.index((26,3)))
to_run_list.pop(to_run_list.index((23,1)))
to_run_list.pop(to_run_list.index((23,2)))
to_run_list.pop(to_run_list.index((23,3)))

# # sequential:
# for i in to_run_list:
#     mask_data_and_fix_header(i)

# MP
with mp.Pool(10) as p:
    p.map(mask_data_and_fix_header, to_run_list)

./derivatives/ds-01/fmriprep/sub-05/func/sub-05_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/fmriprep/sub-09/func/sub-09_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/fmriprep/sub-14/func/sub-14_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/fmriprep/sub-11/func/sub-11_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/fmriprep/sub-13/func/sub-13_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


./derivatives/ds-01/fmriprep/sub-08/func/sub-08_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/fmriprep/sub-06/func/sub-06_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


./derivatives/ds-01/fmriprep/sub-12/func/sub-12_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
./derivatives/ds-01/fmriprep/sub-07/func/sub-07_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


...

/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


./derivatives/ds-02/fmriprep/sub-14/func/sub-14_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


./derivatives/ds-02/fmriprep/sub-15/func/sub-15_task-randomdotmotion_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


# 3. Generate fsf-files for each subject
Note that we first manually create a fsf-file using the FSL FEAT GUI, and then simply change all the paths within the generated fsf-file. We set a smoothing kernel of FWHM=5mm, disable all other preprocessing (including high-pass filtering - include cosines in the confounds file instead). The 8 events of interest were included (all using a double-gamma HRF with temporal derivatives), and 15 contrasts were defined:

- 8 basic contrasts:
    1. stim_easy
    2. stim_hard
    3. cue_left
    4. cue_right
    5. cue_neutral
    6. response_left
    7. response_right
    8. error
- `cue`, `stim`, `response`:
    9. cue
    10. stim
    11. response
- The actual constrasts of interest:
    12. Cue payoff - cue neutral (= (cue_left + cue_right) - 2\*cue_neutral)
    13. stim_hard - stim_easy
    14. Response left - response right
    15. Cue left - cue right

(I should really learn how to use nipype for this).

In [9]:
# function to adapt a design file: We give this function the .fsf file that was manually created in the FSL FEAT GUI, 
# the function adapts all relevant parameters to all other participants / runs

def fix_fsf_file(sub, run,
                 orig_fsf='./derivatives/ds-01/feat_files/sub-01/run-1/first_level_model.fsf', 
                 write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    ds, sub = get_sub_n(sub)
    
    # replace output directory
    fsf = fsf.replace('set fmri(outputdir) "/home/stevenm/bias/derivatives/ds-01/feat_files/sub-01/run-1/firstlevel"',
                      'set fmri(outputdir) "/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/run-{}/firstlevel"'.format(str(ds).zfill(2), str(sub).zfill(2), run))
    
    fsf = fsf.replace('sub-01_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold',
                      'sub-{}_task-randomdotmotion_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold'.format(str(sub).zfill(2), str(run).zfill(2)))
    
    # replace directories for func files & ev files
    fsf = fsf.replace("/home/stevenm/bias/derivatives/ds-01/feat_files/sub-01/run-1",
                      "/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/run-{}".format(str(ds).zfill(2), str(sub).zfill(2), run))
    
    # save fsf as txt
    if write_out:
        output_dir = "/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/run-{}".format(str(ds).zfill(2), str(sub).zfill(2), run)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf

    
subjects = np.arange(1, 35)
runs = [1,2,3]

for sub in subjects:
    if sub == 23:
        continue
    if sub == 6 or sub == 26:
        runs = [1,2]
    else:
        runs = [1,2,3]
    for run in runs:
        fix_fsf_file(sub, run)

# 4. Run first-level FEAT

It takes about 6.5 hrs to run a single run on our machine.
In total, there's 97 runs (33 subs x 3 - 2), with 30 cpus in parallel, totalling ~21.01667 hours of runtime

In [ ]:
def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

# find all newly created design files
all_fsfs = sorted(glob.glob('./derivatives/ds-*/feat_files/sub-*/run*/design.fsf'))
all_fsfs

with mp.Pool(30) as p:
    outputs = p.map(run_feat, all_fsfs)

./derivatives/ds-01/feat_files/sub-01/run-1/design.fsf
./derivatives/ds-01/feat_files/sub-02/run-2/design.fsf
./derivatives/ds-01/feat_files/sub-01/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-03/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-03/run-1/design.fsf
./derivatives/ds-01/feat_files/sub-05/run-1/design.fsf
./derivatives/ds-01/feat_files/sub-04/run-2/design.fsf
./derivatives/ds-01/feat_files/sub-05/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-14/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-09/run-2/design.fsf
./derivatives/ds-01/feat_files/sub-08/run-1/design.fsf
./derivatives/ds-01/feat_files/sub-08/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-10/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-07/run-2/design.fsf
./derivatives/ds-01/feat_files/sub-12/run-1/design.fsf
./derivatives/ds-01/feat_files/sub-10/run-1/design.fsf
./derivatives/ds-01/feat_files/sub-12/run-3/design.fsf
./derivatives/ds-01/feat_files/sub-18/run-3/design.fsf
./derivati

## 5. Second-level GLM

In the second level, we combine runs per subject using FEAT OLS.

#### Interim: Fix "registration not run"-issue for fixed effects

Feat higher-order analyses require "registration to have been performed", which is true, but not by FSL so FSL doesn't know. To do this, "trick" FSL into thinking they have been run by linking identity matrices to the feat dirs.

See also https://neurostars.org/t/performing-full-glm-analysis-with-fsl-on-the-bold-images-preprocessed-by-fmriprep-without-re-registering-the-data-to-the-mni-space/784


In [13]:
def link_files(feat_dir, fsl_dir='/usr/share/fsl/5.0'):
    reg_dir = os.path.join(feat_dir, 'reg')
    if not os.path.exists(reg_dir):
        os.makedirs(reg_dir)
    os.system('ln -s {}/etc/flirtsch/ident.mat {}/reg/example_func2standard.mat'.format(fsl_dir, feat_dir))
    os.system('ln -s {}/etc/flirtsch/ident.mat {}/reg/standard2example_func.mat'.format(fsl_dir, feat_dir))
    os.system('ln -s {}/mean_func.nii.gz {}/reg/standard.nii.gz'.format(feat_dir, feat_dir))
    return 0

subjects = np.arange(1, 35)
runs = [1,2,3]

for sub in subjects:
    if sub == 23:
        continue
    if sub == 6 or sub == 26:
        runs = [1,2]
    else:
        runs = [1,2,3]
    
    ds, sub = get_sub_n(sub)
    for run in runs:
        feat_dir = '/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/run-{}/firstlevel.feat'.format(str(ds).zfill(2), str(sub).zfill(2), run)
        if os.path.exists(feat_dir):
            link_files(feat_dir)


# 6. Generate .fsf-files for second-level fixed-effects


In [23]:
# Same reasoning as above: We manually create an FSL FEAT .fsf file for a single subject, and then 
# use the function below to adapt it to the other subjects

def fix_fsf_file_fe(sub, orig_fsf='./derivatives/ds-01/feat_files/sub-01/secondlevel/design.fsf', 
                    write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    orig_sub = sub
    ds, sub = get_sub_n(sub)
    for run in [1,2,3]:
        # replace directories..
        fsf = fsf.replace("/home/stevenm/bias/derivatives/ds-01/feat_files/sub-01/run-{}/firstlevel.feat".format(run),
                          "/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/run-{}/firstlevel.feat".format(str(ds).zfill(2), str(sub).zfill(2), run))
    
    # output directory
    fsf = fsf.replace('set fmri(outputdir) "/home/stevenm/bias/derivatives/ds-01/feat_files/sub-01/secondlevel"',
                      'set fmri(outputdir) "/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/secondlevel"'.format(str(ds).zfill(2), str(sub).zfill(2)))
    
    # mask
    fsf = fsf.replace('set fmri(threshmask) "/home/stevenm/bias/derivatives/ds-01/fmriprep/sub-01/func/sub-01_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"',
                      'set fmri(threshmask) "/home/stevenm/bias/derivatives/ds-{}/fmriprep/sub-{}/func/sub-{}_task-randomdotmotion_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"'.format(str(ds).zfill(2), str(sub).zfill(2), str(sub).zfill(2)))
    
    # For subject 6 and 26, remove run 3 (no data collected, subject stopped due to fatigue)
    if orig_sub == 6 or orig_sub == 26:
        # change n inputs
        fsf = fsf.replace("set fmri(npts) 3", "set fmri(npts) 2")
        fsf = fsf.replace("set fmri(multiple) 3", "set fmri(multiple) 2")
        
        # change feat directories
        fsf = fsf.replace('\n# 4D AVW data or FEAT directory (3)', '')
        fsf = fsf.replace('\nset feat_files(3) "/home/stevenm/bias/derivatives/ds-{}/feat_files/sub-{}/run-3/firstlevel.feat"'.format(str(ds).zfill(2), str(sub).zfill(2)), '')
        
        # change EV specification
        fsf = fsf.replace('\n# Higher-level EV value for EV 1 and input 3', '')
        fsf = fsf.replace('\nset fmri(evg3.1) 1.0', '')
        fsf = fsf.replace('\n# Group membership for input 3', '')
        fsf = fsf.replace('\nset fmri(groupmem.3) 1', '')
        
    # save fsf as txt
    if write_out:
        output_dir = './derivatives/ds-{}/feat_files/sub-{}/secondlevel'.format(str(ds).zfill(2), str(sub).zfill(2))
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
#fix_fsf_file_fe('01', 'me', write_out=False)

subjects = np.arange(1, 35)
                      
for sub in subjects:
    if sub == 23:
        continue
    else:
        fix_fsf_file_fe(sub, write_out=True)

In [ ]:
## run again

def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

# find all newly created design files
all_fsfs = sorted(glob.glob('./derivatives/ds-*/feat_files/sub-*/secondlevel/design.fsf'))
#all_fsfs.pop(1)

with mp.Pool(30) as p:
    outputs = p.map(run_feat, all_fsfs)

./derivatives/ds-01/feat_files/sub-03/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-04/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-06/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-07/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-11/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-02/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-09/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-08/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-14/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-12/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-01/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-13/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-16/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-05/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-17/secondlevel/design.fsf
./derivatives/ds-01/feat_files/sub-10/secondlevel/design.fsf
./derivatives/ds-01/feat

## Third-level .fsf made manually

The following inputs were used

In [25]:
ls = sorted(glob.glob('/home/stevenm/bias/derivatives/ds-*/feat_files/sub*/secondlevel.gfeat/cope1.feat'))
print('\n'.join(ls))

/home/stevenm/bias/derivatives/ds-01/feat_files/sub-01/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-02/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-03/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-04/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-05/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-06/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-07/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-08/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-09/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-10/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-11/secondlevel.gfeat/cope1.feat
/home/stevenm/bias/derivatives/ds-01/feat_files/sub-12/secondlevel.gfeat/cop

The third-level design matrix includes 5 regressors: Intercept, dataset dummy, and parametric covariates for difficulty-related `v_shift`, cue-based bias-related `z_shift` and `v_shift_bias`

In [4]:
all_parameters_dmc = pd.read_csv('./derivatives/BMAParameters.csv')

design_matrix = pd.DataFrame([1] * all_parameters_dmc.shape[0], columns=['intercept'])
design_matrix['dataset'] = [-1] * 19 + [1] * 14
design_matrix['z_shift'] = all_parameters_dmc['zshiftBias'].values
design_matrix['v_shift'] = all_parameters_dmc['vshift'].values
design_matrix['v_shift_bias'] = all_parameters_dmc['vshiftBias'].values
design_matrix['z_shift'] -= design_matrix['z_shift'].mean()            # demean
design_matrix['v_shift'] -= design_matrix['v_shift'].mean()            # demean
design_matrix['v_shift_bias'] -= design_matrix['v_shift_bias'].mean()  # demean

print(design_matrix.to_string(index=False))

 intercept  dataset   z_shift   v_shift  v_shift_bias
         1       -1 -0.004981 -0.177194     -0.264739
         1       -1 -0.022742 -0.485131     -0.055483
         1       -1  0.024262 -0.375284     -0.175741
         1       -1 -0.027583 -0.779270      1.081206
         1       -1 -0.008379 -0.111900     -0.081597
         1       -1 -0.056664 -0.050581      0.712415
         1       -1 -0.039258 -0.271502     -0.134774
         1       -1 -0.041889 -0.608545      0.338863
         1       -1  0.029381 -0.309642     -0.187815
         1       -1  0.011050 -0.086109     -0.124718
         1       -1 -0.068963 -0.772153     -0.130902
         1       -1 -0.002430  0.120058     -0.023091
         1       -1 -0.025744 -0.569487     -0.196138
         1       -1  0.050135  0.006094     -0.118774
         1       -1 -0.068406  0.678613     -0.147604
         1       -1  0.050824 -0.615191      0.869406
         1       -1 -0.071715 -0.428689      0.542869
         1       -1  0.09617

In [5]:
design_matrix.corr()

,intercept,dataset,z_shift,v_shift,v_shift_bias
intercept,NaN,NaN,NaN,NaN,NaN
dataset,NaN,1.000000,0.250401,0.610567,-0.354717
z_shift,NaN,0.250401,1.000000,0.273428,-0.359327
v_shift,NaN,0.610567,0.273428,1.000000,-0.509534
v_shift_bias,NaN,-0.354717,-0.359327,-0.509534,1.000000


In [15]:
# After manually making a third-level FSF for the first COPE, we generate third-level .fsf-files for the other COPEs

def fix_fsf_file_flame1(cope,
                        orig_fsf='./derivatives/both/glm_level3/third_level_design_dmc.fsf', 
                        write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    # output directory
    fsf = fsf.replace("/home/stevenm/bias/derivatives/both/glm_level3/cope1",
                      "/home/stevenm/bias/derivatives/both/glm_level3/cope{}".format(cope))
    
    for ds in [1,2]:
        for sub in np.arange(1,20):
            fsf = fsf.replace('/home/stevenm/bias/derivatives/ds-0{}/feat_files/sub-{}/secondlevel.gfeat/cope1.feat'.format(ds, str(sub).zfill(2)),
                              '/home/stevenm/bias/derivatives/ds-0{}/feat_files/sub-{}/secondlevel.gfeat/cope{}.feat'.format(ds, str(sub).zfill(2), cope))
        
    # save fsf as txt
    if write_out:
        output_dir = '/home/stevenm/bias/derivatives/both/glm_level3/fsfs/cope{}'.format(cope)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
copes = np.arange(1, 16)

for cope in copes:
    fix_fsf_file_flame1(cope=cope, write_out=True)

In [ ]:
def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

# find all newly created design files
all_fsfs = sorted(glob.glob('./derivatives/both/glm_level3/fsfs/cope*/design.fsf'))
all_fsfs

with mp.Pool(16) as p:
    outputs = p.map(run_feat, all_fsfs)

./derivatives/both/glm_level3/fsfs/cope1/design.fsf
./derivatives/both/glm_level3/fsfs/cope7/design.fsf
./derivatives/both/glm_level3/fsfs/cope10/design.fsf
./derivatives/both/glm_level3/fsfs/cope11/design.fsf
./derivatives/both/glm_level3/fsfs/cope12/design.fsf
./derivatives/both/glm_level3/fsfs/cope13/design.fsf
./derivatives/both/glm_level3/fsfs/cope14/design.fsf
./derivatives/both/glm_level3/fsfs/cope6/design.fsf
./derivatives/both/glm_level3/fsfs/cope8/design.fsf
./derivatives/both/glm_level3/fsfs/cope15/design.fsf
./derivatives/both/glm_level3/fsfs/cope9/design.fsf
./derivatives/both/glm_level3/fsfs/cope5/design.fsf
./derivatives/both/glm_level3/fsfs/cope2/design.fsf
./derivatives/both/glm_level3/fsfs/cope4/design.fsf
./derivatives/both/glm_level3/fsfs/cope3/design.fsf


that's it